In [1]:
credit = read.csv('../../R_projects//MLwR//Machine Learning with R (2nd Ed.)//Chapter 05//credit.csv')

In [2]:
library(caret)

Loading required package: lattice
Loading required package: ggplot2


In [3]:
modelLookup('C5.0')

model,parameter,label,forReg,forClass,probModel
C5.0,trials,# Boosting Iterations,FALSE,TRUE,TRUE
C5.0,model,Model Type,FALSE,TRUE,TRUE
C5.0,winnow,Winnow,FALSE,TRUE,TRUE


In [4]:
set.seed(300)

In [5]:
m = train(default ~ ., data = credit, method = 'C5.0')

Loading required package: C50
Loading required package: plyr


In [6]:
m

C5.0 

1000 samples
  16 predictor
   2 classes: 'no', 'yes' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 1000, 1000, 1000, 1000, 1000, 1000, ... 
Resampling results across tuning parameters:

  model  winnow  trials  Accuracy   Kappa    
  rules  FALSE    1      0.6847204  0.2578421
  rules  FALSE   10      0.7112829  0.3094601
  rules  FALSE   20      0.7221976  0.3260145
  rules   TRUE    1      0.6888432  0.2549192
  rules   TRUE   10      0.7113716  0.3038075
  rules   TRUE   20      0.7233222  0.3266866
  tree   FALSE    1      0.6769653  0.2285102
  tree   FALSE   10      0.7222552  0.2880662
  tree   FALSE   20      0.7297858  0.3067404
  tree    TRUE    1      0.6771020  0.2219533
  tree    TRUE   10      0.7173312  0.2777136
  tree    TRUE   20      0.7285714  0.3058474

Accuracy was used to select the optimal model using  the largest value.
The final values used for the model were trials = 20, model = tree and winnow
 = FALSE.

1. A brief description of the input dataset.
2. A report of the preprocessing and resampling methods applied
3. A list of the candidate models evaluated: 12 different models were tested based on the combinations of three C5.0 tuning parameters
4. The choice of the best model

In [7]:
p = predict(m, credit)

In [8]:
table(p, credit$default)

     
p      no yes
  no  700   2
  yes   0 298

In [9]:
head(predict(m, credit))

[1] no  yes no  no  yes no 
Levels: no yes

In [10]:
head(predict(m, credit, type = 'prob'))

no,yes
0.9606970,0.03930299
0.1388444,0.86115561
1.0000000,0.00000000
0.7720279,0.22797208
0.2948062,0.70519385
0.8583715,0.14162851


## choosing the resampling method

trainControl() is used to create a set of configuration options known as a control object, which guides the train() function.

In [11]:
ctrl = trainControl(method = 'cv', number = 10, 
                   selectionFunction = 'oneSE')

## choosing the grid of parameters to optimize

In [12]:
grid = expand.grid(.model = 'tree',
                  .trials = c(1, 5, 10, 15, 20, 25, 30, 35),
                  .winnow = "FALSE")

In [13]:
grid

.model,.trials,.winnow
tree,1,FALSE
tree,5,FALSE
tree,10,FALSE
tree,15,FALSE
tree,20,FALSE
tree,25,FALSE
tree,30,FALSE
tree,35,FALSE


In [14]:
set.seed(300)

In [15]:
m = train(default ~ ., data = credit, method = "C5.0",
         metric = 'Kappa',
         trControl = ctrl,
         tuneGrid = grid)

Warning message in Ops.factor(x$winnow):
“‘!’ not meaningful for factors”

In [16]:
m

C5.0 

1000 samples
  16 predictor
   2 classes: 'no', 'yes' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results across tuning parameters:

  trials  Accuracy  Kappa    
   1      0.724     0.3124461
   5      0.713     0.2921760
  10      0.719     0.2947271
  15      0.721     0.3009258
  20      0.717     0.2929875
  25      0.728     0.3150336
  30      0.729     0.3104144
  35      0.741     0.3389908

Tuning parameter 'model' was held constant at a value of tree
Tuning
 parameter 'winnow' was held constant at a value of FALSE
Kappa was used to select the optimal model using  the one SE rule.
The final values used for the model were trials = 1, model = tree and winnow
 = FALSE.

## improving model performance with meta-learning (teamwork-based techniques / ensembles)

+ better generalizability to future problems
+ improved performance on massive or miniscule datasets
+ ability to synthesize data from distinct domains
+ a more nuanced understanding of difficult learning tasks

## Bagging

Uses a technique called bootstrap aggregating or bagging. Bagging generates a number of training datasets by bootstrap sampling the original data. These datasets are then used to generate a set of models using a single learning algorithm. Predictions are combined using voting or averaging (for numeric prediction).

In [17]:
library('ipred')

In [18]:
set.seed(300)
mybag = bagging(default ~ ., data = credit, nbagg = 25)

In [19]:
credit_pred = predict(mybag, credit)

In [20]:
table(credit_pred, credit$default)

           
credit_pred  no yes
        no  699   2
        yes   1 298

In [21]:
set.seed(300)
ctrl = trainControl(method = 'cv', number = 10)
train(default ~ ., data = credit, method = 'treebag', trControl = ctrl)

Loading required package: e1071


Bagged CART 

1000 samples
  16 predictor
   2 classes: 'no', 'yes' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results:

  Accuracy  Kappa    
  0.747     0.3618416


In [22]:
str(svmBag)

List of 3
 $ fit      :function (x, y, ...)  
 $ pred     :function (object, x)  
 $ aggregate:function (x, type = "class")  


In [23]:
svmBag$fit

function (x, y, ...) 
{
    loadNamespace("kernlab")
    out <- kernlab::ksvm(as.matrix(x), y, prob.model = is.factor(y), 
        ...)
    out
}
<environment: namespace:caret>

In [24]:
bagctrl = bagControl(fit = svmBag$fit,
                    predict = svmBag$pred,
                    aggregate = svmBag$aggregate)

In [26]:
library(caret)
library(ipred)

In [28]:
set.seed(300)
svmbag <- train(default ~ ., data = credit, "bag",
                trControl = ctrl, bagControl = bagctrl)

Warning message in data.row.names(row.names, rowsi, i):
“some row.names duplicated: 3,6,8,9,10,13,15,17,22,27,28,32,33,36,37,39,42,43,45,47,50,51,55,57,59,60,63,64,65,67,69,73,78,80,86,89,95,100,104,110,112,118,120,121,124,128,129,132,134,135,138,139,140,144,150,151,154,155,157,162,165,166,167,169,175,177,179,180,182,190,194,195,201,203,207,209,213,215,216,218,220,222,225,227,229,233,235,238,241,243,245,246,248,252,254,255,264,268,273,274,275,279,280,281,286,287,289,291,293,295,297,300,302,306,311,315,319,320,322,324,326,327,334,338,341,342,348,349,351,352,356,358,360,363,369,370,372,374,377,378,384,386,388,393,394,396,398,400,403,404,405,407,410,413,414,420,422,425,427,428,435,440,443,445,447,448,450,452,454,459,461,464,465,466,468,469,471,473,475,476,478,480,481,486,487,489,490,492,493,498,501,502,507,514,516,518,522,523,529,536,542,546,548,551,552,554,557,559,561,563,566,569,570,571,574,575,581,582,584,586,587,590,591,593,600,603,605,607,610,612,613,616,618,620,622,623,627,629,633,6

“some row.names duplicated: 5,8,9,13,15,17,18,20,22,24,25,27,28,31,33,36,37,41,43,45,47,48,51,57,58,60,66,68,69,72,73,75,76,79,80,82,83,87,91,93,94,96,99,104,105,106,109,110,111,112,114,116,117,120,123,124,131,132,133,140,143,145,146,147,152,159,163,166,167,173,178,179,186,192,194,196,199,205,207,209,212,216,219,220,223,224,226,229,236,238,241,242,244,246,250,253,256,258,261,263,264,265,271,272,276,280,287,288,291,298,300,301,302,304,306,310,313,316,317,320,321,322,324,331,332,334,336,338,339,345,346,349,357,358,360,362,365,368,369,373,379,385,390,394,397,398,401,403,405,407,408,411,416,418,420,421,424,428,432,433,436,438,439,443,444,445,448,454,455,459,461,463,465,473,475,478,485,496,498,501,504,510,512,515,516,518,522,525,526,527,530,532,533,535,538,539,547,548,553,556,558,560,561,563,565,568,569,570,572,575,576,579,581,582,583,585,586,589,596,599,600,602,603,608,610,612,616,620,623,625,626,629,631,632,635,639,640,646,647,649,656,663,666,670,672,677,681,683,686,687,688,689,693,694,69

“some row.names duplicated: 2,9,12,13,15,16,18,19,21,26,28,33,35,41,43,47,51,53,56,58,60,65,66,68,71,72,77,79,80,82,83,86,89,94,95,98,104,107,108,109,112,114,119,124,126,129,130,132,134,136,137,138,141,142,143,145,146,148,152,154,155,157,160,164,165,170,171,173,175,178,179,181,184,186,187,190,192,196,197,198,202,208,210,212,214,216,217,218,220,222,224,226,230,233,235,237,241,243,245,247,248,249,250,253,255,256,259,262,268,269,271,275,276,277,279,282,283,287,289,292,294,296,297,298,303,304,309,312,313,317,318,319,324,331,337,344,347,352,355,358,360,362,366,370,372,373,376,380,381,385,389,397,399,400,404,406,408,413,414,417,421,431,432,433,437,443,445,448,451,452,461,462,464,465,467,469,472,473,476,481,485,488,491,492,494,502,505,506,508,509,510,514,515,516,520,522,523,529,530,531,533,535,542,544,548,549,551,552,561,562,570,577,578,580,581,585,587,589,591,592,593,595,596,599,601,608,610,613,616,618,622,624,625,626,630,632,635,637,640,641,646,648,649,651,653,656,658,659,663,665,667,668,67

“some row.names duplicated: 2,3,8,10,12,14,15,16,19,20,22,23,25,27,32,33,37,38,40,41,42,43,45,49,51,52,53,56,59,64,66,69,74,76,78,82,86,87,88,90,91,93,95,97,99,104,113,119,121,123,124,129,135,138,139,141,143,147,149,152,154,155,160,162,164,166,168,169,173,174,175,177,180,181,189,190,192,193,195,197,200,205,207,210,214,217,218,221,224,225,231,236,238,241,242,243,246,250,251,253,258,260,262,265,266,267,270,273,275,277,279,280,281,282,283,289,291,293,295,297,298,301,302,304,305,306,310,312,313,319,322,326,330,332,334,336,339,340,341,343,345,346,348,349,351,354,358,361,362,363,365,366,368,370,375,377,380,383,385,392,393,396,398,399,400,402,403,404,407,411,413,415,417,423,424,427,430,433,435,438,441,442,446,447,453,458,463,464,467,471,473,478,481,486,489,493,494,499,501,502,511,513,515,518,520,521,524,530,535,539,548,549,550,554,558,560,561,564,565,567,572,573,575,578,584,587,592,594,595,596,599,601,602,603,609,620,621,623,624,628,629,634,636,637,639,641,642,644,645,650,652,654,658,662,666,

“some row.names duplicated: 2,4,5,20,21,23,32,33,38,40,41,43,44,48,51,53,55,57,61,65,67,68,72,74,76,77,79,80,82,86,89,91,92,94,105,110,115,119,120,122,125,129,132,135,137,138,143,145,147,152,154,156,159,161,163,166,167,168,171,172,173,174,177,178,179,181,184,192,194,197,202,203,205,213,214,216,218,223,227,228,232,236,238,240,243,244,246,248,256,257,260,261,264,266,267,269,273,274,276,278,282,284,285,288,296,297,298,299,301,302,303,305,306,314,319,320,325,326,330,331,337,338,340,343,350,351,353,360,361,362,364,370,371,373,375,377,379,380,385,387,390,391,392,394,398,400,402,403,404,408,409,411,421,422,425,428,431,433,435,437,438,440,442,445,453,457,458,459,461,462,466,471,476,479,480,487,488,492,493,495,496,497,500,501,508,510,513,516,520,522,524,525,528,536,538,539,543,545,547,548,557,558,560,561,563,566,567,569,571,574,575,577,580,586,590,593,596,601,602,603,605,608,609,611,613,616,620,621,625,626,627,630,631,638,639,641,642,645,646,649,650,654,655,657,660,663,665,668,672,678,681,683,6

...

“some row.names duplicated: 2,4,7,9,11,12,16,17,19,20,23,25,27,32,33,35,36,38,40,41,43,45,49,53,55,56,64,66,67,70,73,76,78,82,84,86,90,95,97,100,105,107,109,111,114,115,120,122,124,125,129,131,132,134,135,136,139,143,145,149,156,160,163,169,174,175,177,180,185,187,194,195,197,199,204,205,206,208,210,214,218,219,221,222,223,228,230,233,236,237,240,243,245,250,252,253,262,264,266,270,273,274,275,278,279,282,285,290,297,298,302,305,306,307,309,311,314,316,318,321,322,329,336,338,340,341,343,347,349,350,352,361,365,366,367,369,372,374,383,384,385,389,390,391,395,396,403,404,407,408,411,413,414,416,420,424,427,429,430,432,440,443,445,449,451,452,454,457,460,462,466,467,471,474,477,478,479,482,485,490,491,492,495,497,502,504,509,510,514,516,523,524,528,530,533,534,535,540,541,542,546,547,548,550,554,558,560,564,565,566,569,579,580,583,584,587,590,592,593,604,605,609,612,615,616,625,626,628,629,630,632,635,638,641,642,643,644,651,654,660,662,664,669,676,681,683,686,688,690,691,693,697,702,705

“some row.names duplicated: 3,6,8,9,14,17,19,21,22,24,26,29,31,33,36,45,47,48,50,52,56,61,63,67,69,70,72,74,75,78,82,84,88,89,97,99,101,103,106,111,114,119,123,125,127,133,137,138,142,146,148,151,153,163,165,167,169,171,172,177,183,189,194,197,199,200,206,207,209,210,214,216,217,219,220,223,224,225,227,229,232,234,236,237,240,244,248,252,253,256,257,259,261,262,263,266,272,276,278,280,283,287,293,296,302,304,307,310,311,313,314,321,325,328,330,331,337,341,345,346,348,350,351,352,355,357,362,363,364,366,367,368,373,374,376,377,379,381,383,385,388,392,398,402,407,409,414,418,422,427,431,432,433,435,437,438,442,445,449,452,457,460,462,466,469,475,480,485,486,487,488,495,499,501,503,505,509,512,513,515,517,518,520,521,524,525,527,529,531,533,534,536,537,539,541,546,549,550,553,556,558,563,565,566,569,574,576,577,578,582,583,584,591,592,595,596,600,603,605,607,610,613,614,616,618,620,623,625,630,633,634,640,644,654,657,662,666,673,676,677,679,680,682,683,684,686,688,691,696,701,704,706,710,

“some row.names duplicated: 2,5,6,10,11,15,16,19,21,23,26,28,33,37,38,40,41,44,45,47,49,51,53,59,63,64,67,71,72,74,75,78,79,82,83,85,87,89,91,92,97,99,101,103,105,107,108,113,118,122,123,128,129,133,134,136,139,147,148,152,156,158,159,163,165,166,168,169,175,178,179,183,188,190,193,194,199,202,209,213,214,220,222,227,229,231,233,239,241,247,248,251,257,258,260,261,263,264,266,267,270,273,278,281,284,291,293,300,301,305,306,307,310,312,313,319,322,324,325,326,327,332,333,336,338,344,346,348,350,352,353,355,359,363,366,368,369,370,372,375,377,379,380,382,388,392,395,396,398,399,401,402,412,413,415,417,419,421,422,424,429,430,433,436,438,442,443,454,456,459,462,465,468,470,473,475,477,482,484,485,486,489,490,494,495,498,500,502,507,508,511,513,515,516,521,523,530,533,534,541,545,546,548,550,551,553,556,560,564,568,569,571,572,575,577,578,583,586,600,604,606,608,609,611,612,613,615,616,617,619,620,621,623,627,628,630,632,633,635,641,645,648,649,651,652,657,659,661,662,668,672,675,679,680,6

Something is wrong; all the Accuracy metric values are missing:
    Accuracy       Kappa    
 Min.   : NA   Min.   : NA  
 1st Qu.: NA   1st Qu.: NA  
 Median : NA   Median : NA  
 Mean   :NaN   Mean   :NaN  
 3rd Qu.: NA   3rd Qu.: NA  
 Max.   : NA   Max.   : NA  
 NA's   :1     NA's   :1    


ERROR: Error: Stopping


In [29]:
svmbag

ERROR: Error in eval(expr, envir, enclos): object 'svmbag' not found


Error :(

## boosting

Similar to bagging, boosting uses ensembles of models trained on resampled data and a vote to determine the final prediction.
+ Resampled datasets in boosting are constructed specifically to generate complementary learners
+ Rather than giving each learner an equal vote, boosting gives each learner's vote a weight based on its past performance

Beginning from an unweighted dataset, the first classifier attempts to model the outcome. Examples that the classifier predicted correctly will be less likely to appear in the training dataset for the following classifier, and conversely, the difficult-to-classify examples will appear more frequently. As additional rounds of weak learners are added, they are trained on data with successively more difficult examples.

Boosting is most commonly used for decision trees.

In [32]:
library(adabag)

Loading required package: rpart
Loading required package: mlbench

Attaching package: ‘adabag’

The following object is masked from ‘package:ipred’:

    bagging



In [33]:
set.seed(300)
m_adaboost = boosting(default ~ ., data = credit)

In [34]:
p_adaboost = predict(m_adaboost, credit)

In [35]:
head(p_adaboost$class)

[1] "no"  "yes" "no"  "no"  "yes" "no"

In [36]:
p_adaboost$confusion

               Observed Class
Predicted Class  no yes
            no  700   0
            yes   0 300

In [37]:
set.seed(300)
adaboost_cv = boosting.cv(default ~ ., data = credit)

i:  1 Thu Jun  8 11:45:46 2017 
i:  2 Thu Jun  8 11:45:52 2017 
i:  3 Thu Jun  8 11:45:59 2017 
i:  4 Thu Jun  8 11:46:05 2017 
i:  5 Thu Jun  8 11:46:11 2017 
i:  6 Thu Jun  8 11:46:18 2017 
i:  7 Thu Jun  8 11:46:24 2017 
i:  8 Thu Jun  8 11:46:30 2017 
i:  9 Thu Jun  8 11:46:37 2017 
i:  10 Thu Jun  8 11:46:43 2017 


In [38]:
adaboost_cv$confusion

               Observed Class
Predicted Class  no yes
            no  594 151
            yes 106 149

In [39]:
library(vcd)

Loading required package: grid


In [40]:
Kappa(adaboost_cv$confusion)

            value    ASE     z  Pr(>|z|)
Unweighted 0.3607 0.0323 11.17 5.914e-29
Weighted   0.3607 0.0323 11.17 5.914e-29

## random forests

Random Forests combine versatility and power into a single machine learning approach. As the ensemble uses only a small, random portion of the full feature set, random forests can handle extremely large datasets, where the so-called "curse of dimensionality" might cause other models to fail.

+ An all-purpose model that performs well on most problems
+ Can handle noisy or missing data as well as categorical or continuous features
+ Selects only the most important features
+ Can be used on data with an extremely large number of features or examples
- Unline a decision tree, the model is not easily interpretable
- May require some work to tune the model to the data

In [41]:
# randomForests(train, class, ntree, mtry)
# train = data frame containing training data
# class = factor vector with the class for each row in the training data
# ntree = number of trees to grow
# mtry = number of features to randomly select at each split

In [45]:
library('randomForest')

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin



In [48]:
set.seed(300)
rf = randomForest(default ~ ., data = credit)

In [49]:
rf


Call:
 randomForest(formula = default ~ ., data = credit) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 4

        OOB estimate of  error rate: 23.8%
Confusion matrix:
     no yes class.error
no  640  60  0.08571429
yes 178 122  0.59333333

Trees included: 500, tried 4 variables at each split.

out-of-bag error, which unlike the resubstitution error, is an unbiased estimate of the test set error. It should be a fairly reasonable estimate of future perfomance.

### evaluating RF peformance

In [50]:
ctrl = trainControl(method = 'repeatedcv', number = 10, repeats = 10)

In [51]:
grid_rf = expand.grid(.mtry = c(2, 4, 8, 16))

In [53]:
set.seed(300)
m_rf = train(default ~ ., data = credit, method = 'rf',
            metric = "Kappa", trControl = ctrl, tuneGrid = grid_rf)

In [54]:
grid_c50 = expand.grid(.model = 'tree',
                      .trials = c(10, 20, 30, 40),
                      .winnow = 'FALSE')

In [55]:
set.seed(300)
m_c50 = train(default ~ ., data = credit, method = 'C5.0',
             metric = 'Kappa', trControl = ctrl, tuneGrid = grid_c50)

Warning message in Ops.factor(x$winnow):
“‘!’ not meaningful for factors”

In [56]:
m_rf

Random Forest 

1000 samples
  16 predictor
   2 classes: 'no', 'yes' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 10 times) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results across tuning parameters:

  mtry  Accuracy  Kappa    
   2    0.7256    0.1311283
   4    0.7476    0.2878470
   8    0.7519    0.3346061
  16    0.7557    0.3618152

Kappa was used to select the optimal model using  the largest value.
The final value used for the model was mtry = 16.

In [57]:
m_c50

C5.0 

1000 samples
  16 predictor
   2 classes: 'no', 'yes' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 10 times) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results across tuning parameters:

  trials  Accuracy  Kappa    
  10      0.7325    0.3215655
  20      0.7343    0.3268052
  30      0.7381    0.3343137
  40      0.7388    0.3335082

Tuning parameter 'model' was held constant at a value of tree
Tuning
 parameter 'winnow' was held constant at a value of FALSE
Kappa was used to select the optimal model using  the largest value.
The final values used for the model were trials = 30, model = tree and winnow
 = FALSE.

With a kappa of about .361, the random forest model with mtry = 16 was the winner among these eight models. It was higher than the best C5.0 decision tree, which had a kappa of about .334, and slightly higher than the AdaBoost.M1 model with a kappa of about .36. Based on these results we would submit the random forest as our final model.